# Выбор локации для скважины

Для добывающей компании «ГлавРосГосНефть» нужно решить, где бурить новую скважину.

Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Нужно построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Также требуется проанализировать возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

Описание данных:
- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в скважине (тыс. баррелей).

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [2]:
filepath_list = ['datasets/geo_data_0.csv', 'datasets/geo_data_1.csv', 'datasets/geo_data_2.csv']

In [3]:
def prepering_data(filepath):
    #Прочитаем данные из файла
    data = pd.read_csv(filepath)
    
    print('Первые несколько строк данных:')
    display(data.head())
    
    print('Есть ли неопределенные данные:')
    display(pd.DataFrame(round(data.isna().mean()*100,1)).style.background_gradient('coolwarm'))
    
    print('Есть ли дубликаты в данных:')
    display(data.duplicated().sum())
    
    #Столбец id не влияет на целевой признак. Поэтому исключим его из признаков:
    data = data.drop(['id'], axis=1)
    
    print("\033[4m")
    print('Матрица корреляции данных')
    print("\033[0m")    
    display(data.corr())
    
    
    return data

In [4]:
data = []
index = 0
for filepath in filepath_list:
    print('\033[91m\033[1m')
    print(f'Регион {index}:')
    print('\033[90m\033[0m')
    index += 1

    data.append( prepering_data(filepath) )


Регион 0:

Первые несколько строк данных:


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


Есть ли неопределенные данные:


,0
id,0.000000
f0,0.000000
f1,0.000000
f2,0.000000
product,0.000000


Есть ли дубликаты в данных:


...

,0
id,0.000000
f0,0.000000
f1,0.000000
f2,0.000000
product,0.000000


Есть ли дубликаты в данных:


0


Матрица корреляции данных



,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


**Выводы.** 

*Региона 0:* 
- По первым строкам данных можно сделать вывод, что потребуется промасштабировать признаки перед обучением модели.
- Данные не содержат пропусков и дубликатов.
- Корреляционная матрица данных, показывает небольшую связь целевого признака *product* и признаков *f0*, *f1*, равную 14,4% и 19,2% соответственно. Связь признака *f2* с целевым признаком более значительная - 48%. Связь признаков *f0*, *f2* с целевым признаком прямопропорциональная, а признака *f1* обратнопропорциональная.

*Региона 1:* 
- По первым строкам данных можно сделать вывод, что потребуется промасштабировать признаки перед обучением модели.
- Данные не содержат пропусков и дубликатов.
- Корреляционная матрица данных, показывает отсутствие связи целевого признака *product* и признаков *f0*, *f1*, корреляция равна 3% и 1% соответственно. Связь признака *f2* с целевым признаком более значительная - 99%. Связь признака *f2* прямопропорциональная.

*Региона 2:* 
- По первым строкам данных можно сделать вывод, что потребуется промасштабировать признаки перед обучением модели.
- Данные не содержат пропусков и дубликатов.
- Корреляционная матрица данных, показывает отсутствие связи целевого признака *product* и признаков *f0*, *f1*, корреляция равна 0,2% и 0,1% соответственно. Связь признака *f2* с целевым признаком более значительная - 44,6%. Связь признака *f2* обратнопропорциональная.

In [5]:
#Функция для разделения данных на тестовую и валидационную выборку
def spliter(data, target_name):
    train, valid = np.split(data.sample(frac=1, random_state=821), [int(.75*len(data))])
    
    #Отделим признаки и целевой признак в выборках. Сбросим индексы выборок
    features_train = train.drop(target_name, axis=1).reset_index(drop=True)
    target_train = train[target_name].reset_index(drop=True)

    features_valid = valid.drop(target_name, axis=1).reset_index(drop=True)
    target_valid = valid[target_name].reset_index(drop=True)
    
    #Убедимся, что разбиение было выполнено верно, оценим размеры выборок.
    print('Размер тренировочной выборки:', features_train.shape[0])
    print('Размер валидационной выборки:', features_valid.shape[0])
    
    return features_train, target_train, features_valid, target_valid

In [6]:
features_train_list = []
target_train_list = []
features_valid_list = []
target_valid_list = []
index = 0
for region_data in data:
    print('\033[91m\033[1m')
    print(f'Регион {index}:')
    print('\033[90m\033[0m')
    index += 1
    features_train, target_train, features_valid, target_valid = \
        spliter(data=region_data, target_name='product')
    
    #Масштабируем данные
    numeric = ['f0', 'f1', 'f2']
    scaler = StandardScaler()
    scaler.fit(features_train[numeric])
    features_train[numeric] = scaler.transform(features_train[numeric])
    features_valid[numeric] = scaler.transform(features_valid[numeric])
    
    #Сохраним данные в список
    features_train_list.append(features_train)
    target_train_list.append(target_train)
    features_valid_list.append(features_valid)
    target_valid_list.append(target_valid)


Регион 0:

Размер тренировочной выборки: 75000
Размер валидационной выборки: 25000

Регион 1:

Размер тренировочной выборки: 75000
Размер валидационной выборки: 25000

Регион 2:

Размер тренировочной выборки: 75000
Размер валидационной выборки: 25000


## Обучение и проверка модели

In [7]:
model = LinearRegression()

predictions_valid_list = []
index = 0
for features_train, target_train, features_valid, target_valid in \
        zip(features_train_list, target_train_list, features_valid_list, target_valid_list):
    print('\033[91m\033[1m')
    print(f'Регион {index}:')
    print('\033[90m\033[0m')
    index += 1
    
    model.fit(features_train, target_train)
    
    predictions_valid = model.predict(features_valid)
    predictions_valid_list.append( pd.Series(predictions_valid) )
    result = mean_squared_error(target_valid, predictions_valid) ** 0.5
    print("RMSE модели линейной регрессии на валидационной выборке:", result)
    
    print('Средний запас предсказанного сырья', predictions_valid.mean())
    
    
    #Проверка адекватности модели
    predicted_valid = pd.Series(target_train.mean(), index=target_valid.index)
    result = mean_squared_error(target_valid, predicted_valid) ** 0.5
    print("RMSE константной модели:", result)


Регион 0:

RMSE модели линейной регрессии на валидационной выборке: 37.729963027045024
Средний запас предсказанного сырья 92.75021438728606
RMSE константной модели: 44.307568280856835

Регион 1:

RMSE модели линейной регрессии на валидационной выборке: 0.8922537395970659
Средний запас предсказанного сырья 68.86820255182714
RMSE константной модели: 46.09712179750057

Регион 2:

RMSE модели линейной регрессии на валидационной выборке: 40.21264523908406
Средний запас предсказанного сырья 94.97609258225137
RMSE константной модели: 44.8085263125068


**Вывод.** Минимальное значение RMSE на валидационной выборки у модели обученной на данных региона 1

## Подготовка к расчёту прибыли

In [8]:
#Стоимость барреля, руб.
COST_BARREL = 450
#Бюджет на разработку скважин, руб.
COSTS_BUDGET = 10e9
#стоимость 1000 баррелей, руб
COST_1000_BARREL = COST_BARREL * 1000

**Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины**

In [9]:
#Количество разрабатываемых скважин равно 200.
#Поэтому бюджет на разработку одной скважины равен:
ONE_WELL_BUDGET = COSTS_BUDGET / 200

#Поделив бюджет разработки одной скважины на стоимость барреля
#получим минимальный объём сырья для безубыточной разработки новой скважины
VOLUME = ONE_WELL_BUDGET / COST_BARREL

#Т.к. столбец product в данных представлен в тысячах баррелях, то для удобности сравнения
#разделим объем на 1000
VOLUME_1000_BARREL = VOLUME / 1000

print('Минимальный объём сырья для безубыточной разработки новой скважины:', VOLUME_1000_BARREL)

Минимальный объём сырья для безубыточной разработки новой скважины: 111.11111111111111


**Вывод.** Минимальный объём сырья для безубыточной разработки новой скважины превышает среднее значение предсказанного сырья в скажине каждого из регионов.

## Расчёт прибыли и рисков 

In [10]:
#Функция расчета прибыли
def income(target, probabilities, count):
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    return ( selected.sum() * COST_1000_BARREL ) - COSTS_BUDGET

In [11]:
#Важно данную запись делать в отдельной ячейки. Иначе выборки не получаются случайные, при повторном запуски ячейки
STATE = np.random.RandomState(12345)

In [12]:
def bootstrap_for_income(target, predictions):
    values = []   
    for i in range(1000):
        target_subsample = target.sample(500, replace=True, random_state=STATE)
        probs_subsample = predictions[target_subsample.index]
        revenue_score = income(target=target_subsample, probabilities=probs_subsample, count=200)
        values.append(revenue_score)
    
    values = pd.Series(values)
    lower = values.quantile(q=0.025)
    high = values.quantile(q=0.975)
    risk_of_loss = values[values < 0].count() / values.count() * 100
    
    mean = values.mean()
    print("Средняя прибыль:", mean)
    print("2,5%-квантиль:", lower)
    print("97,5%-квантиль:", high)
    print(f'Риск убытков: {risk_of_loss}%')

In [13]:
index = 0
for target_valid, predictions_valid in zip(target_valid_list, predictions_valid_list):  
    print('\033[91m\033[1m')
    print(f'Регион {index}:')
    print('\033[90m\033[0m')
    index += 1
    bootstrap_for_income(target_valid, predictions_valid)


Регион 0:

Средняя прибыль: 500294566.7104109
2,5%-квантиль: -29186278.874776587
97,5%-квантиль: 1019433824.7981983
Риск убытков: 3.5999999999999996%

Регион 1:

Средняя прибыль: 531341051.6062054
2,5%-квантиль: 108824178.20255733
97,5%-квантиль: 939386325.2297403
Риск убытков: 0.5%

Регион 2:

Средняя прибыль: 383677490.9059474
2,5%-квантиль: -181337923.78597215
97,5%-квантиль: 949148325.1957636
Риск убытков: 7.8%


**Вывод.** *Регион 1* имеет большую среднюю прибыль, чем  *Регион 0* и *Регион 2*. А также единственным регионом, в котором вероятность убытков меньше 2.5% (около 0,5%). Поэтому для разработки скважин, предлагается *Регион 1*. 